# Statyczne typy
Typowanie jest w rzeczywistości nakładaniem ograniczeń na zbiory wartości zmiennych w programach, co ułatwia wnioskowanie o nich, w szczególności w sposób automatyczny. Istnieją różne podejścia do tego problemu i różne sposoby jego kategoryzacji - jednym z nich jest to kiedy typowanie się odbywa. Ze względu na tą cechę, języki mogą być statycznie lub dynamicznie typowane.

Statyczne typy są sprawdzane przed wykonaniem, co ułatwia konstruowanie narzędzi weryfikujących poprawność programów przed ich rzeczywistym wykonaniem. Historycznie patrząc, języki statycznie typowane były nieco trudniejsze do pisania dla programistów - np.: zmienne nie mogły zmieniać swojego typu w trakcie wykonania, ale narzędzia diagnozujące problemy przed wykonaniem były nieco bardziej zaawansowane, podobnie jak optymalizacje, których mógł dokonać kompilator, w rezultacie produkując szybszy kod.

Amerykańska korporacja AirBnB przeprowadziła analizę postmortem, z której wynikało, że 38% bugów mogłoby być unikniętych, jeśli postawiono by na język typowany statycznie (w ich przypadku porównanie dotyczyło JavaScript vs TypeScript).

W Pythonie oryginalnie postawiono na typowanie dynamiczne - interpreter kalkuluje i sprawdza typy w czasie wykonania, a zmienne mogą swobodnie zmieniać swoje typy w trakcie działania programu. Kod, który nigdy nie będzie wykonany, nie jest typowany:

In [ ]:
if False:
    1 + "Hello"
print("world")

In [ ]:
1 + "Hello"

In [ ]:
2 * "Hello"

In [ ]:
test = "test"
print(type(test))
test = 5
print(type(test))

Wraz z rozwojem języka, Python stał się językiem o opcjonalnym typowaniu statycznym (*ang: gradually typed* - przy użyciu anotacji typów) i pojawiły się narzędzia wymuszające statyczne typowanie - np.: `mypy`. Mimo to, zdecydowana większość kodu Pythonowego nadal jest dynamicznie typowana, a język nie wymusza stosowania anotacji typowych, choć należy podkreślić benefity ich stosowania, jak np.: wykrywanie wielu bugów przy użyciu narzędzi statycznej analizy kodu.

### Deklarowanie typów

#### Funkcje
```
def func(arg: arg_type, optarg: arg_type = default) -> return_type:
    ...
    
def func2(x: str) -> Optional[str]:
    if x != "":
        return "cokolwiek" + x
    else:
        return None
```

#### Zmienne
```
cośtam: str
jakaś_liczba: int
cards: list[Card]
results: dict[tuple[int, int, int], str]

from typing import Union
lst: list[Union[int, str, bool, Card]]
```

### Mypy
Aby użyć `mypy` wystarczy, że najpierw zainstalujemy go przez `pip`:
```
pip install mypy
```

... a potem odpalimy przy użyciu:
```
mypy --check-untyped-defs my_script.py
```
Flaga `--check-untyped-defs` włącza sprawdzanie typów w ciele nieotypowanych funkcji - domyślnie sprawdzany jest tylko scope globalny. Można również wymusić sprawdzanie czy wszystkie funkcje są otypowane - przy użyciu `--disallow-untyped-defs`.

Przykładowo dla takiego wejścia:

```
def multiply_by_2(number: float) -> float:
    return 2 * number


multiply_by_2("I'm not a float")
```

dostaniemy:
```
>>> mypy float_check.py
float_check.py:5: error: Argument 1 to "multiply_by_2" has incompatible
                        type "str"; expected "float"
Found 1 error in 1 file (checked 1 source file)
```
Przydatne do debugowania błędów znalezionych przez `mypy` bywa użycie dwóch funkcji: `reveal_type` oraz `reveal_locals`. Początkowo były zdefiniowane przez `mypy`, jednak od wersji 3.11 `reveal_type` jest dostępne do zaimportowania z modułu typing biblioteki standardowej. `reveal_locals` musi zostać usunięte po wykonaniu `mypy`, inaczej będzie skutkować błędem.

```
from typing import reveal_type, reveal_locals
import math
reveal_type(math.e)

radius = 1
area = math.pi * radius * radius
reveal_locals()
```

```
>>> mypy reveal.py
reveal.py:4: error: Revealed type is 'builtins.float'

reveal.py:8: error: Revealed local types are:
reveal.py:8: error: area: builtins.float
reveal.py:8: error: radius: builtins.int
```
Oczywiście, `mypy` ma dużo innych opcji i konfiguracji - ich opis można znaleźć [tu](https://mypy.readthedocs.io/en/stable/).


### Konfiguracja mypy
Mypy możemy konfigurować albo podając opcje w linii komend, albo z użyciem plików. Następujące pliki są sprawdzane:
```
./mypy.ini
./.mypy.ini
./pyproject.toml
./setup.cfg
$XDG_CONFIG_HOME/mypy/config
~/.config/mypy/config
~/.mypy.ini
```
Plik konfiguracyjny ma standardową składnię plików `ini` i wygląda przykładowo tak:
```

[mypy]
warn_return_any = True
warn_unused_configs = True

# Per-module options:

[mypy-mycode.foo.*]
disallow_untyped_defs = True

[mypy-mycode.bar]
warn_return_any = False

[mypy-somelibrary]
ignore_missing_imports = True
```
Dokładna dokumentacja mypy znajduje się pod [tym linkiem](https://mypy.readthedocs.io/en/stable/)

### *Zadanie*
W katalogu `examples` znajduje się skrypt `image_blur.py`. Użyj `mypy --strict` by znaleźć w programie błędy i popraw je aż mypy przestanie zgłaszać jakiekolwiek zastrzeżenia. Pobierz z internetu dowolny obrazek, uruchom program podając go za parametr skryptu i przetestuj, czy działa po Twoich poprawkach.

### Kiedy w ogóle przejmować się typami

1. Kiedy ma się małe doświadczenie z Pythonem, to zdecydowanie **nie jest konieczne** - może pomóc, ale są istotniejsze mechanizmy języka
2. W drobnych, podręcznych skryptach, które nie będą obiektem zainteresowań reszty zespołu w firmie - **raczej nie warto**. W niczym nie przeszkadzają, IDE/edytor może lepiej podpowiadać, ale nie ma to wielkiego znaczenia
3. **Zdecydowanie warto** w bibliotekach używanych przez innych, zwłaszcza w tych, którymi dzielicie się przez PyPI (lub lokalną jego instalację). Wówczas użytkownicy mogą zdecydowanie łatwiej poznawać waszą bibliotekę z pomocę ich IDE, bez konieczności zgadywania waszych intencji i supresowania błędów typów podkreślanych przez ich edytor
4. **Zdecydowanie warto** w jakkolwiek większych projektach, gdzie jeden lub więcej zespołów kolaborują nad kodem przez dłuższy okres czasu - wówczas dużo łatwiej zrozumieć założenia architektoniczne oryginalnych autorów, używać innych modułów i znajdować subtelne bugi.


Gdy chcemy się zajmować typami w Pythonie i używać ich to ulepszania struktury naszej aplikacji, istnieje kilka mechanizmów, które pozwalają nam w elastyczny sposób z nimi pracować, nieco podobnie do języków statycznie typowanych: `dataclasses`, `ABC` i `Protocols`.

## ABC
Abstrakcyjne klasy bazowe to klasy, które istnieją po to by być rozszerzanymi przez dziedziczenie. Definiują one abstrakcyjny interfejs, który służy za swojego rodzaju kontrakt na którym może polegać reszta kodu. Abstrakcyjne klasy bazowe nie mają w założeniu ich twórców kiedykolwiek służyć do tworzenia obiektów - część ich metod może nie być zaimplementowana! Teoretycznie nic nie stoi na przeszkodzie bo owe klasy tworzyć tak:

In [ ]:
# abstrakcyjna klasa bazowa
class UserRepository:
    def fetch(id):
        raise NotImplementedError
    
    def store(user):
        raise NotImplementedError

# rzeczywista implementacja
class SQLiteUserRepository(UserRepository):
    def __init__(self, path):
        with open(path) as db:
            self.db = connect(db)
            ... # tu jakieś otwarcie bazy
        
    def fetch(id):
        row = self.db.execute(f"SELECT * FROM auth_user WHERE id = {sanitize(id)}")
        ... # tu jakaś dalsza część logiki
    
    def store(user):
        self.db.execute(f"INSERT INTO ...")

def func1(user: User, repository: UserRepository):
    # modyfikacje na user
    repository.store(user)

... jednak nie mamy żadnej gwarancji, że ktoś nie spróbuję stworzyć egzemplarza naszej abstrakcyjnej klasy bazowej. Z pomocą przychodzi moduł `abc` biblioteki standardowej:

In [4]:
from abc import ABC, abstractmethod

class UserRepository(ABC):
    @abstractmethod
    def fetch(id):
        ...
    
    @abstractmethod
    def store(user):
        ...

class InMemoryUserRepository(UserRepository):
    def store(user):
        ...

    def fetch(id):
        ...

# repo = UserRepository()
repo = InMemoryUserRepository()

Abstrakcyjne klasy bazowe w Pythonie mają znaczenie głównie w większych bazach kodu, gdzie nawigowanie grafu współpracujących obiektów jest dość trudne i przydaje się mechanizm opisujący na jakich abstrakcjach polegamy. Takie do tego podejście jest przykładem tzw. nominalnego podejścia do typowania - klasa jest uznawana za spełniającą określony interfejs wtedy, gdy eksplicite odziedziczy po klasie abstrakcyjnej (lub jej podklasie) mającej taki interfejs.

Implementowanie wielu klas abstrakcyjnych, zwłaszcza, gdy ich interfejsy nie mają wspólnego przecięcia, jest również jednym z sensownych zastosowań wielodziedziczenia.

## Protocol - [PEP-544](https://www.python.org/dev/peps/pep-0544/)
Z punktu widzenia statycznego typowania Pythona, nominalne typowanie - np.: przy użyciu `abc.ABC` pozwala na określenie naszych oczekiwań względem konkretnej implementacji interfejsu na którym chcemy polegać. Wtedy wiemy, że możemy bezpiecznie wywoływać metody lub odczytywać property przekazanego nam obiektu (oczywiście, jeśli uruchomimy analizator statyczny - np.: `mypy`), bez narażania się na błędy podczas wykonania kodu na produkcji.

To podejście zaczyna jednak być bardzo uciążliwe, gdy mamy wiele współpracujących pakietów - łatwo wówczas np.: o cykle w importach. Dodatkowo, ten model myślenia o typach jest dość nieelastyczny i wymaga przestawienia myślenia na inne niż w przypadku dynamicznego typowania, w przypadku, gdy jesteśmy przyzwyczajeni do zapominania o typach w Pythonie. Duck-typing, z którego jest znany Python i który w rzeczywistości działa podczas wykonania kodu polega na obserwacji, że "jeśli coś kwacze jak kaczka, chodzi jak kaczka to jest kaczką". Inaczej mówiąc interesuje nas to, że dany obiekt ma określone zachowania i dane, nie zaś szczegóły tego, jak został zaimplementowany i czy jego twórca w ogóle wiedział o naszym przypadku użycia.

W związku z tym problemem PEP-544 wprowadził do świata Pythona koncept `Protocols`, który umożliwia spełnianie relacji bycia podtypem, przez klasy, które nie definiują explicite, że dziedziczą po danym interfejsie, ale implementują jego metody i wymagane property:

In [ ]:
from typing import Protocol

# Protokół
class SupportsClose(Protocol):
    def close(self) -> None:
        ...

# Przykładowa implementacja - nie musi dziedziczyć po SupportsClose
class Resource:
    def __init__(self, f):
        self.file = f

    def close(self) -> None:
        self.file.close()

# ...ale może
class Connection(SupportsClose):
    def close():
        print("closing connection")
        ...

# Przykład użycia w sygnaturze typu
def close_all(things: Iterable[SupportsClose]) -> None:
    for t in things:
        t.close()

f = open('foo.txt')
r = Resource(open("bar.txt"))
close_all([f, r])  # Oba OK!
close_all([1])     # Error: 'int' has no 'close' method

Protokoły mogą też definiować wymagane pola - jednak muszą być zgodne z [PEP-526](https://www.python.org/dev/peps/pep-0526):

In [ ]:
from typing import Protocol, List

class Template(Protocol):
    name: str        # This is a protocol member
    value: int = 0   # This one too (with default)

    def method(self) -> None:
        self.temp: List[int] = [] # Error in type checker

class Concrete:
    def __init__(self, name: str, value: int) -> None:
        self.name = name
        self.value = value

    def method(self) -> None:
        return

var: Template = Concrete('value', 42)  # OK

Protokoły mogą rozszerzać się przez (wielo-)dziedziczenie, ale nie mogą dziedziczyć po klasach, które nie są protokołami:

In [ ]:
from typing import Sized, Protocol

class SizedAndClosable(Sized, Protocol):
    def close(self) -> None:
        ...

## Aliasy i nowe typy z już istniejących

### Aliasy
W celu uproszczenia długich i nieczytelnych anotacji typowych (np.: `Callable[[int, int], int]`) możemy wprowadzić alias typu:

In [7]:
from typing import Callable

BinaryOp = Callable[[int, int], int]

def calcWithoutAlias(arg1: int, arg2: int, op: Callable[[int, int], int]) -> int:
    print("calculating...")
    return op(arg1, arg2)

def calcWithAlias(arg1: int, arg2: int, op: BinaryOp) -> int: # nawet sama sygnatura niesie więcej informacji o intencjach autora
    print("calculating...")
    return op(arg1, arg2)


W Pythonie 3.12 zmienia się sposób definiowania aliasów - należy użyć słowa kluczowego `type`:
```
type BinaryOp = Callable[[int, int], int]
```

### NewType
Czasami chcemy wprowadzić typ, nazywający inny, już istniejący, ale w przeciwieństwie do aliasów nie chcemy pozwolić na traktowanie ich wymiennie. Wówczas należy posłużyć się konstrukcją `NewType`:

In [8]:
from typing import NewType

UserId = NewType('UserId', int)
some_id = UserId(524313)

def get_user_name(user_id: UserId) -> str:
    ...

# działa, nie ma błędu w statycznych checkerach
user_a = get_user_name(UserId(42351))

# też działa, ale statyczne checkery zwracają błędy
user_b = get_user_name(-1)

ważne: tak stworzonych typów nie można używać jako klas bazowych!

## Typy generyczne
Czasami implementuje się algorytmy dzialajace na dowolnych sekwencjach, niezależnie od typów w nich przechowywanych:

In [10]:
from collections.abc import Sequence
from typing import TypeVar

T = TypeVar('T')

def take(s: Sequence[T], num_elems: int):
    return s[:num_elems]

Większość kontenerów w Pythonie ma anotacje typowe parametryzowane jedną zmienną typową np.: `list[T]`. W przypadku, gdy zawierają jednak więcej typów, możemy skorzystać z unii: `list[int | str]` lub w ogóle dopuścić dowolny typ przy użyciu `Any`: `list[Any]`. Wyjątkami są `tuple`, które mogą być parametryzowane dowolną ilością zmiennych typowych, np.: `tuple[str, int, bool]`. Wynika to z częstego używania tupli jako sposobu na zwrócenie wielu wartości z funkcji, gdzie zazwyczaj wartości są różnych typów. Jeśli wiemy, że każdy element tupli jest tego samego typu możemy napisać `tuple[int, ...]`

### Generyczne klasy
Klasy definiowane przez użytkownika mogą również być parametryzowane zmiennymi typowymi, o ile wśród klas po których dziedzicą jest klasa `Generic`, która bierze dowolnie dużo różnych zmiennych typowych jako swoje parametry:

In [11]:
from typing import TypeVar, Generic
from logging import Logger
from collections.abc import Iterable

T = TypeVar('T')

class LoggedVar(Generic[T]):
    def __init__(self, value: T, name: str, logger: Logger) -> None:
        self.name = name
        self.logger = logger
        self.value = value

    def set(self, new: T) -> None:
        self.log('Set ' + repr(self.value))
        self.value = new

    def get(self) -> T:
        self.log('Get ' + repr(self.value))
        return self.value

    def log(self, message: str) -> None:
        self.logger.info('%s: %s', self.name, message)


def zero_all_vars(vars: Iterable[LoggedVar[int]]) -> None:
    for var in vars:
        var.set(0)

### TypeVar
Definiując zmienne typowe, posługujemy się klasą `TypeVar`, która umożliwia definiowanie ograniczeń na typy, które można w jej miejsce podstawiać:

In [13]:
from typing import TypeVar
T = TypeVar('T')  # można podstawić dowolny typ
S = TypeVar('S', bound=str)  # tylko podtypy str mogą być podstawiane za S
A = TypeVar('A', str, bytes)  # tylko str albo bytes są akceptowane zamiast A

## Przykłady anotacji wspierających wykrywanie nietrywialnych błędów

### @override - dostępne od Pythona 3.12
Podobnie jak w Javie czy C# możemy oznaczyć metody przedefiniowywane w podklasach dekoratorem `@override`, by ułatwić statycznym checkerom wykrywanie błędów, gdy metoda nadklasy uległa zmianie, ale miejsca wywołania gdzie podstawiona jest podklasa nie. Nie jest to obowiązkowe, ale może pomóc:

In [ ]:
from abc import ABC, abstractmethod
from typing import override

class Base(ABC):
    @abstractmethod
    def foo(self, x) -> None:
        ...

class Derived(Base):
    @override
    def foo(self) -> None:
        print("derived")

### @final
Ten dekorator można użyć zarówno na metodach jak i klasach. Użyte na metodzie uniemożliwia jej przedefiniowanie w podklasie. Użycie na klasie uniemożliwia dziedziczenie po niej:

In [ ]:
class Base:
    @final
    def done(self) -> None:
        ...
class Sub(Base):
    def done(self) -> None: # błąd podczas statycznej analizy
        ...

@final
class Leaf:
    ...
class Other(Leaf): # błąd podczas statycznej analizy
    ...

## Typeshed i pakiety types-*
Historycznie Python nie miał typów, które można było analizować statycznie. Oczywiście wewnątrz interpretera podczas działania programu typy zawsze odgrywały taką samą rolę jak teraz, jednak ich wartość była znana tylko podczas wykonania. Ta zaszłość historyczna powoduje, że do dziś nie wszystkie moduły biblioteki standardowej czy też po prostu popularne biblioteki zostały przepisane tak, by wspierać anotacje typowe. To zaś powoduje, że statyczne analizatory takie jak `mypy` muszą same definiować sygnatury typów dla tych bibliotek. Służy do tego moduł `typeshed`, który można zainstalować pipem razem z `mypy`. Dodatkowo, niektóre biblioteki mają osobne pakiety w PyPI, które definiują ich typy - należy ich szukać po nazwach `types-{nazwa biblioteki}`. Po ich doinstalowaniu `mypy` nie powinien więcej zwracać ostrzeżeń czy błędów.